# Setup

In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
import copy
from scipy.interpolate import CubicSpline
import torch.optim as optim
import torch.nn as nn
import torch
from torch.optim import Adam
from torch.optim import Optimizer
from scipy.fftpack import fft, ifft
from scipy.stats import mode
from torch.utils.data import DataLoader, TensorDataset
import datetime
from sklearn.metrics import f1_score
from sklearn.cluster import AgglomerativeClustering
import warnings

## Hyperparameters

In [63]:
seed = 420
num_epochs = 20
batch_size = 32  # Set your batch size
learning_rate_client = 0.001
local_epochs = 1
subject_dir = 'FL_Data/windowed_data_refused_5aug/subject_'  # Set your directory to the subject data
num_clients = 54
num_classes = 4 # number of transformations
local_learning_rate = 0.01
local_steps= 1
learning_rate_decay= False
future_test= False
learning_rate_decay_gamma= 0.99
n_clusters= 4
mu= 1
global_rounds= 100
join_ratio= 1.0
eval_gap= 1
detailed_info= False
partition= "nature"
initial_rounds= 10
metric= 'cosine'
linkage= 'complete'

#current timestamp
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [3]:
# set the seed
torch.manual_seed(420)
np.random.seed(420)

# Load Data

In [4]:
def load_data_client(id, batch_size=batch_size, type='labelled_train'):
    # Load the data
    data = np.load(subject_dir + str(id) + '/windowed_' + type + '_x.npy')
    labels = np.load(subject_dir + str(id) + '/windowed_' + type + '_y.npy')

    # print shape of data
    # print(data.shape)
    # print(labels.shape)

    # Convert to torch tensor
    data = torch.from_numpy(data).float()
    labels = torch.from_numpy(labels).long()

    # Create a dataset
    dataset = torch.utils.data.TensorDataset(data, labels)

    # Create a dataloader
    if type == 'labelled_train' or type == 'unlabelled_train':
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    else:
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    # dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    return dataloader

# Optimizers

In [5]:
class PerturbedGradientDescent(Optimizer):
  def __init__(self, params, lr=0.01, mu=0.0):
    default = dict(lr=lr, mu=mu)
    super().__init__(params, default)

  @torch.no_grad()
  def step(self, global_params, device):
    for group in self.param_groups:
      for p, g in zip(group['params'], global_params):
        g = g.to(device)
        d_p = p.grad.data + group['mu'] * (p.data - g.data)
        p.data.add_(d_p, alpha=-group['lr'])

# Model Architecture

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Train and Test Function

In [7]:
# def train_autoencoder(model, train_loader, device, learning_rate=0.01, epochs=5):
#     model.to(device)
#     criterion = nn.MSELoss()
#     optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
#     model.train()
#     total_loss = 0
#     for epoch in range(epochs):
#         for data, target in train_loader:
#             data, target = data.to(device), target.to(device)
#             # print(data.shape)
#             data = data.permute(0, 2, 1)
#             optimizer.zero_grad()
#             output = model(data)
#             # print(output.shape)
#             loss = criterion(output, data)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
        
#         epoch_loss = total_loss / len(train_loader)
#         # print(f'Epoch {epoch+1}, Loss: {epoch_loss}')
#         total_loss = 0  # Reset total loss for the next epoch

#     results = {
#         'train_loss': epoch_loss
#     }
    
#     return results  # Returns the average loss of the last epoch

In [8]:
# def test_autoencoder(model, test_loader, device):
#     model.to(device)
#     model.eval()
    
#     criterion = nn.MSELoss()
#     total_loss = 0
    
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             data = data.permute(0, 2, 1)
#             output = model(data)
#             loss = criterion(output, data)
#             total_loss += loss.item()
    
#     avg_loss = total_loss / len(test_loader)
#     # print(f'Test Loss: {avg_loss}')
    
#     return avg_loss  # Returns the average loss for the test data

In [9]:
# function to train the model
def train_model(model, train_loader, device, learning_rate=0.001, epochs=1):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    model.train()
    total_loss = 0
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        epoch_loss = total_loss / len(train_loader)
        total_loss = 0  # Reset total loss for the next epoch

    results = {
        'train_loss': epoch_loss
    }
    
    return results  # Returns the average loss of the last epoch

In [10]:
# function to test the model
# method to test the model and get the accuracy and f1 score
def test_model(model, test_loader):
    model.to(device)
    model.eval()
    test_num = 0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            #calculate test_num
            test_num += len(target)

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            y_true.extend(target.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    accuracy = correct / total
    f1 = f1_score(y_true, y_pred, average='weighted')
    # print(f'Accuracy: {accuracy}, F1 Score: {f1}')

    return correct, total

# Client

## Client Base

In [11]:
class Client(object):
  """
  Base class for clients in federated learning.
  """

  def __init__(self, model, id, **kwargs):
    self.model = copy.deepcopy(model)
    self.device = device
    self.id = id
    self.num_classes = num_classes
    self.batch_size = batch_size
    self.learning_rate = local_learning_rate
    self.local_steps = local_steps
    self.data_path = subject_dir
    self.learning_rate_decay = learning_rate_decay
    self.future_test = future_test


    # check BatchNorm
    self.has_BatchNorm = False
    for layer in self.model.children():
      if isinstance(layer, nn.BatchNorm2d):
        self.has_BatchNorm = True
        break

    self.loss = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate) # momentum=0.9, weight_decay=1e-4
    self.learning_rate_scheduler = torch.optim.lr_scheduler.ExponentialLR(
      optimizer=self.optimizer,
      gamma=learning_rate_decay_gamma
    )

  def load_train_data(self, batch_size=batch_size):
    train_data = load_data_client(self.id, batch_size, 'unlabelled_train')
    # get length of train data
    self.train_samples = len(train_data)

    return train_data
    
    # if batch_size == None:
    #   batch_size = self.batch_size
    # train_data = read_client_data(self.dataset, self.data_path, self.id, is_train=True)

    # # label poison attack
    # if self.malicious and self.attack_type == 'A1':
    #   for idx in range(len(train_data)):
    #     train_data[idx][1] = self.num_classes - train_data[idx][1] - 1
    # self.train_samples = len(train_data)
    # return DataLoader(train_data, batch_size, drop_last=True, shuffle=False)

  def load_test_data(self, batch_size=batch_size):
    test_data = load_data_client(self.id, batch_size, 'unlabelled_train')

    return test_data
    # """
    # fine-tunes the model using the loaded training data
    # """
    # if batch_size == None:
    #   batch_size = self.batch_size
    # test_data = read_client_data(self.dataset, self.data_path, self.id, is_train=False)
    # return DataLoader(test_data, batch_size, drop_last=False, shuffle=False)

  def set_parameters(self, model):
    for new_param, old_param in zip(model.parameters(), self.model.parameters()):
      old_param.data = new_param.data.clone()

  def test_metrics_personalized(self):
    testloaderfull = self.load_test_data()

    test_acc, test_num = test_model(self.model, testloaderfull)

    # self.model.eval()

    # test_acc = 0
    # test_num = 0
    # y_prob = []
    # y_true = []

    # with torch.no_grad():
    #   for x, y in testloaderfull:
    #     if type(x) == type([]):
    #       x[0] = x[0].to(self.device)
    #     else:
    #       x = x.to(self.device)
    #     y = y.to(self.device)
    #     output = self.model(x)

    #     test_acc += (torch.sum(torch.argmax(output, dim=1) == y)).item()
    #     test_num += y.shape[0]

    #     y_prob.append(output.detach().cpu().numpy())
    #     nc = self.num_classes
    #     if self.num_classes == 2:
    #       nc += 1
    #     lb = label_binarize(y.detach().cpu().numpy(), classes=np.arange(nc))
    #     if self.num_classes == 2:
    #       lb = lb[:, :2]
    #     y_true.append(lb)

    # y_prob = np.concatenate(y_prob, axis=0)
    # y_true = np.concatenate(y_true, axis=0)

    return test_acc, test_num

  def train_metrics_personalized(self):
    trainloader = self.load_train_data()

    self.model.eval()

    train_num = 0
    losses = 0
    with torch.no_grad():
      for x, y in trainloader:
        x = x.to(self.device)
        y = y.to(self.device)
        x = x.permute(0, 2, 1)
        output = self.model(x)
        loss = self.loss(output, y)
        train_num += y.shape[0]
        losses += loss.item() * y.shape[0]

    return losses, train_num

## Client CHAR

In [12]:
class clientCHAR(Client):
  def __init__(self, model, id, **kwargs):
    super().__init__(model, id, **kwargs)
    self.mu = mu
    self.model_per = copy.deepcopy(self.model)
    self.optimizer_per = PerturbedGradientDescent(self.model_per.parameters(), lr=self.learning_rate, mu=self.mu)
    self.learning_rate_scheduler_per = torch.optim.lr_scheduler.ExponentialLR(
        optimizer=self.optimizer_per,
        gamma=learning_rate_decay_gamma
        )

  def dtrain(self):
    trainloader = self.load_train_data()
    model = copy.deepcopy(self.model)
    self.model.train()
    self.model_per.train()

    max_local_steps = self.local_steps

    for step in range(max_local_steps):
      for x, y in trainloader:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        y = y.to(self.device)
        x = x.permute(0, 2, 1)
        out_p = self.model_per(x)
        loss = self.loss(out_p, y)
        self.optimizer_per.zero_grad()
        loss.backward()
        self.optimizer_per.step(model.parameters(), self.device)

        out_g = self.model(x)
        loss = self.loss(out_g, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    if self.learning_rate_decay:
      self.learning_rate_scheduler.step()
      self.learning_rate_scheduler_per.step()

  def test_metrics_personalized(self):
    testloaderfull = self.load_test_data()

    test_acc, test_num = test_model(self.model_per, testloaderfull)
    # self.model_per.eval()

    # test_acc = 0
    # test_num = 0
    # y_prob = []
    # y_true = []

    # with torch.no_grad():
    #   for x, y in testloaderfull:
    #     if type(x) == type([]):
    #       x[0] = x[0].to(self.device)
    #     else:
    #       x = x.to(self.device)
    #     y = y.to(self.device)
    #     output = self.model_per(x)

    #     test_acc += (torch.sum(torch.argmax(output, dim=1) == y)).item()
    #     test_num += y.shape[0]

    #     y_prob.append(F.softmax(output).detach().cpu().numpy())
    #     y_true.append(label_binarize(y.detach().cpu().numpy(), classes=np.arange(self.num_classes)))

    # y_prob = np.concatenate(y_prob, axis=0)
    # y_true = np.concatenate(y_true, axis=0)

    return test_acc, test_num

  def train_metrics_personalized(self):
    trainloader = self.load_train_data()
    self.model_per.eval()

    train_num = 0
    losses = 0
    with torch.no_grad():
      for x, y in trainloader:
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        x = x.permute(0, 2, 1)
        y = y.to(self.device)
        output = self.model_per(x)
        loss = self.loss(output, y)

        #add a regularization term to the loss
        # ensure that the personalized model doesn't deviate too far from the global model.
        # The strength of this regularization is controlled by the parameter self.mu
        gm = torch.cat([p.data.view(-1) for p in self.model.parameters()], dim=0)
        pm = torch.cat([p.data.view(-1) for p in self.model_per.parameters()], dim=0)
        loss += 0.5 * self.mu * torch.norm(pm-gm, p=2) #element-wise difference using L2 norm

        train_num += y.shape[0]
        losses += loss.item() * y.shape[0]

    return losses, train_num

  def get_update(self, global_model):
    trainloader = self.load_train_data()
    model = copy.deepcopy(self.model) #old model
    self.set_parameters(global_model)
    self.model.train()

    max_local_steps = self.local_steps

    for step in range(max_local_steps):
      for i, (x, y) in enumerate(trainloader):
        # if type(x) == type([]):
        #   x[0] = x[0].to(self.device)
        # else:
        #   x = x.to(self.device)
        x = x.to(self.device)
        x = x.permute(0, 2, 1)
        y = y.to(self.device)
        output = self.model(x)
        loss = self.loss(output, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    model_update = [c_param.data - s_param.data for c_param, s_param in zip(self.model.parameters(), global_model.parameters())]
    self.set_parameters(model)
    return model_update

# Server

## Server Base

In [13]:
class Server(object):
  def __init__(self, model):
    # Set up the main attributes
    self.device = device
    self.num_classes = num_classes
    self.global_rounds = global_rounds
    self.local_steps = local_steps
    self.batch_size = batch_size
    self.learning_rate = local_learning_rate
    self.global_model = copy.deepcopy(model)
    self.num_clients = num_clients
    self.join_ratio = join_ratio
    # self.attack_ratio = attack_ratio
    # self.attack_type = attack_type
    self.seed = seed
    # self.algorithm = algorithm
    self.current_round = -1
    self.future_test = future_test
    # self.future_ratio = future_ratio
    self.num_training_clients = num_clients
    self.join_clients = num_clients
    # self.finetune_rounds = finetune_rounds
    self.eval_gap = eval_gap
    self.detailed_info = detailed_info
    self.partition = partition
    self.data_path = subject_dir

    self.clients = []
    self.training_clients = []
    self.selected_clients = []

    self.uploaded_weights = []
    self.uploaded_ids = []
    self.uploaded_models = []
    self.uploaded_updates = []

    self.rs_test_acc_g = []
    self.rs_train_loss_g = []
    self.rs_test_accs_g = []
    self.rs_test_acc_p = []
    self.rs_train_loss_p = []
    self.rs_test_accs_p = []
    self.ft_train_loss = []
    self.ft_test_acc = []
    self.ft_std_acc = []

  def set_clients(self, model, clientObj):

    if self.future_test == False:
      for i in range(self.num_clients):
        client = clientObj(model=model, id=i)
        self.clients.append(client)

      self.training_clients = self.clients
      self.training_clients_ids = np.arange(self.num_clients)

  def select_clients(self):
    selected_clients = list(np.random.choice(self.training_clients, self.join_clients, replace=False))
    return selected_clients

  def send_models(self):
    for client in self.selected_clients:
      client.set_parameters(self.global_model)

  def receive_models(self):
    self.uploaded_ids = []
    self.uploaded_weights = [] #weight based on the fraction of client's data
    self.uploaded_models = []

    tot_samples = 0
    for client in self.selected_clients:
      tot_samples += client.train_samples
      self.uploaded_ids.append(client.id)
      self.uploaded_weights.append(client.train_samples)
      self.uploaded_models.append(client.model)

    for i, w in enumerate(self.uploaded_weights):
      self.uploaded_weights[i] = w / tot_samples


  def evaluate_personalized(self, acc=None, loss=None):
    stats = self.test_metrics_personalized()
    stats_train = self.train_metrics_personalized()

    # if self.malicious_ids != []:
    #   relative_malicious_ids = np.array([stats[0].index(i) for i in self.malicious_ids])

    #   stats_A = np.array(stats)[:, relative_malicious_ids].tolist()
    #   stats_train_A = np.array(stats_train)[:, relative_malicious_ids].tolist()

    #   test_acc_A = sum(stats_A[2])*1.0 / sum(stats_A[1])
    #   train_loss_A = sum(stats_train_A[2])*1.0 / sum(stats_train_A[1])
    #   accs_A = [a / n for a, n in zip(stats_A[2], stats_A[1])]
    #   losses_A = [a / n for a, n in zip(stats_train_A[2], stats_train_A[1])]

    # else:
    #   test_acc_A = -1
    #   train_loss_A = -1
    #   accs_A = []
    #   losses_A = []

    # benign_ids = np.sort(np.setdiff1d(self.training_clients_ids, self.malicious_ids))
    benign_ids = np.sort(self.training_clients_ids)
    relative_benign_ids = np.array([stats[0].index(i) for i in benign_ids])

    stats_B = np.array(stats)[:, relative_benign_ids].tolist()
    stats_train_B = np.array(stats_train)[:, relative_benign_ids].tolist()

    stats = None
    stats_train = None

    test_acc = sum(stats_B[2])*1.0 / sum(stats_B[1])
    train_loss = sum(stats_train_B[2])*1.0 / sum(stats_train_B[1])
    accs = [a / n for a, n in zip(stats_B[2], stats_B[1])]
    losses = [a / n for a, n in zip(stats_train_B[2], stats_train_B[1])]

    if acc == None:
      self.rs_test_acc_p.append(test_acc)
    else:
      acc.append(test_acc)

    if loss == None:
      self.rs_train_loss_p.append(train_loss)
    else:
      loss.append(train_loss)

    self.rs_test_accs_p.append(accs)

    print("Averaged Train Loss: {:.2f}".format(train_loss))
    print("Averaged Test Accurancy: {:.2f}%".format(test_acc*100))
    print("Std Test Accurancy: {:.2f}%".format(np.std(accs)*100))

  def test_metrics_personalized(self):
    num_samples = []
    tot_correct = []

    for c in self.training_clients:
      ct, ns = c.test_metrics_personalized()
      tot_correct.append(ct*1.0)

      num_samples.append(ns)

    ids = [c.id for c in self.training_clients]
    return ids, num_samples, tot_correct

  def train_metrics_personalized(self):
    num_samples = []
    losses = []
    for c in self.training_clients:
      cl, ns = c.train_metrics_personalized()
      num_samples.append(ns)
      losses.append(cl*1.0)

    ids = [c.id for c in self.training_clients]
    return ids, num_samples, losses

## Server CHAR

In [14]:
class FedCHAR(Server):
  def __init__(self, model):
    super().__init__(model)

    self.set_clients(model, clientCHAR)

    print(f"\nJoin ratio / total clients: {self.join_ratio} / {self.num_training_clients}")
    print("Finished creating server and clients.")

    self.initial_rounds = initial_rounds
    self.n_clusters = n_clusters
    self.metric = metric
    self.linkage = linkage

  def train(self):
    # initial Stage
    for i in range(self.initial_rounds):
      self.selected_clients = self.select_clients()
      self.send_models()

      for client in self.selected_clients:
        client.dtrain()

      if i%self.eval_gap == 0:
        print(f"\n-------------Round number: {i}-------------")
        print("\nEvaluate personalized models for training clients.")
        self.evaluate_personalized()

      self.receive_models()
      self.aggregate_parameters()

    # Clustering Stage
    print(f"\n-------------Clustering-------------")
    clients_updates = self.collect()
    self.cluster_identity = self.cluster(clients_updates)
    cluster_info = [[('Client', idx) for idx, g_id in enumerate(self.cluster_identity) if g_id == i] for i in range(max(self.cluster_identity)+1)]
    for idx, info in enumerate(cluster_info):
      print('Cluster {}: {}'.format(idx, info))

    self.group_models = [copy.deepcopy(self.global_model)] * (max(self.cluster_identity) + 1)

    # Remaining Stage
    for i in range(self.global_rounds - self.initial_rounds):
      self.selected_clients = self.select_clients()
      self.send_models_g()

      for client in self.selected_clients:
        client.dtrain()

      if i%self.eval_gap == 0:
        print(f"\n-------------Round number: {i+self.initial_rounds}-------------")
        print("\nEvaluate personalized models for training clients.")
        self.evaluate_personalized()

      self.receive_models_g()
      self.aggregate_parameters_g(g_epochs=(i+self.initial_rounds))

    print("\nFinal Average Personalized Accuracy: {}\n".format(self.rs_test_acc_p[-1]))

    return self.cluster_identity

  def receive_models(self):
    self.uploaded_ids = []
    self.uploaded_weights = []
    self.uploaded_updates = []

    tot_samples = 0
    for client in self.selected_clients:
      tot_samples += client.train_samples
      self.uploaded_ids.append(client.id)
      self.uploaded_weights.append(client.train_samples)
      self.uploaded_updates.append([c_param.data - s_param.data for c_param, s_param in zip(client.model.parameters(), self.global_model.parameters())])

    for i, w in enumerate(self.uploaded_weights):
      self.uploaded_weights[i] = w / tot_samples

  def add_parameters(self, w, client_update):
    for server_param, client_param in zip(self.global_update, client_update):
      server_param.data += client_param.data.clone() * w

  def aggregate_parameters(self):
    self.global_update = copy.deepcopy(self.uploaded_updates[0])
    for param in self.global_update:
      param.data.zero_()

    for w, client_update in zip(self.uploaded_weights, self.uploaded_updates):
      self.add_parameters(w, client_update)

    for model_param, update_param in zip(self.global_model.parameters(), self.global_update):
      model_param.data += update_param.data.clone()

  def collect(self):
    clients_updates = []
    for client in self.training_clients:
      clients_updates.append(client.get_update(self.global_model))

    clients_updates = [torch.cat([uu.reshape(-1, 1) for uu in u], axis=0).detach().cpu().numpy().squeeze() for u in clients_updates]
    return clients_updates

  def cluster(self, clients_updates):
    clustering = AgglomerativeClustering(n_clusters=self.n_clusters, metric=self.metric, linkage=self.linkage).fit(clients_updates)
    return clustering.labels_

  def send_models_g(self):
    for client in self.selected_clients:
      c_idx = list(self.training_clients_ids).index(client.id)
      client.set_parameters(self.group_models[self.cluster_identity[c_idx]])

  def receive_models_g(self):
    self.uploaded_ids = []
    self.uploaded_weights = []
    self.uploaded_updates = []

    for client in self.selected_clients:
      self.uploaded_ids.append(client.id)
      self.uploaded_weights.append(client.train_samples)
      c_idx = list(self.training_clients_ids).index(client.id)
      self.uploaded_updates.append([c_param.data - s_param.data for c_param, s_param in zip(client.model.parameters(), self.group_models[self.cluster_identity[c_idx]].parameters())])

  def aggregate_parameters_g(self, g_epochs=0):
    for i in range(len(self.group_models)):
      self.global_update = copy.deepcopy(self.uploaded_updates[0])
      for param in self.global_update:
        param.data.zero_()

      user_idx_in_same_group = np.array([r_id for r_id, c_id in enumerate(self.uploaded_ids) if self.cluster_identity[list(self.training_clients_ids).index(c_id)] == i])
      uploaded_weights = [self.uploaded_weights[u_id] for u_id in range(len(self.uploaded_weights)) if u_id in user_idx_in_same_group]
      uploaded_weights = [weight / sum(uploaded_weights) for weight in uploaded_weights]
      uploaded_updates = [self.uploaded_updates[u_id] for u_id in range(len(self.uploaded_updates)) if u_id in user_idx_in_same_group]

      for w, client_update in zip(uploaded_weights, uploaded_updates):
        self.add_parameters(w, client_update)

      for model_param, update_param in zip(self.group_models[i].parameters(), self.global_update):
        model_param.data += update_param.data.clone()

      # save the group_models parameters
      name_path = f'Refused_FL/Model_Global_CHAR_5Aug/{current_time}/Group_{i}'
      # create if name_path is not exist
      if not os.path.exists(name_path):
        os.makedirs(name_path)
      torch.save(self.group_models[i].state_dict(), f"{name_path}/cluster_model_round_{g_epochs}.pth")
      # torch.save(self.group_models[i].state_dict(), f"{name_path}/cluster_model_round_{g_epochs}.pth")

# Main

In [15]:
warnings.simplefilter("ignore")
print("Creating server and clients ...")
start = time.time()
model = CNNFeatureExtractor(num_classes=num_classes).to(device)

print(model)

server = FedCHAR(model)
cluster_member = server.train()
print(f"\nTime cost: {round((time.time()-start)/60, 2)}min.")
print(f"Cluster members: {cluster_member}")
print(f"len cluster member: {len(cluster_member)}")

Creating server and clients ...
CNNFeatureExtractor(
  (conv1): Conv1d(3, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)

Join ratio / total clients: 1.0 / 54
Finished creating server and clients.

-------------Round number: 0-------------

Evaluate personalized models for training clients.
Averaged Train Loss: 1.63
Averaged Test Accurancy: 21.48%
Std Test Accurancy: 2.35%

-------------Round number: 1-------------

Evaluate personalized models for training clients.
Averaged Train Loss: 1.64
Averaged Test Accurancy: 22.50%
Std Test Accurancy: 2.71%

-------------Round number: 2-------------

In [17]:
# save cluster member
np.save(f'Refused_FL/Model_Global_CHAR_5Aug/{current_time}/cluster_member.npy', cluster_member)

# Fine-Tuned Phase

## Model Fine-tuned

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Data Fine-Tuned

### Label Data

In [65]:
labelled_data = []

for i in range(num_clients):
    data_label = load_data_client(id= i, batch_size=batch_size, type='labelled_train')
    print(f"subject id: {i}, len: {len(data_label)}")
    labelled_data.append(data_label)

subject id: 0, len: 11
subject id: 1, len: 10
subject id: 2, len: 11
subject id: 3, len: 10
subject id: 4, len: 12
subject id: 5, len: 12
subject id: 6, len: 2
subject id: 7, len: 3
subject id: 8, len: 1
subject id: 9, len: 10
subject id: 10, len: 8
subject id: 11, len: 11
subject id: 12, len: 10
subject id: 13, len: 12
subject id: 14, len: 10
subject id: 15, len: 9
subject id: 16, len: 12
subject id: 17, len: 11
subject id: 18, len: 10
subject id: 19, len: 11
subject id: 20, len: 10
subject id: 21, len: 10
subject id: 22, len: 8
subject id: 23, len: 11
subject id: 24, len: 1
subject id: 25, len: 1
subject id: 26, len: 1
subject id: 27, len: 1
subject id: 28, len: 1
subject id: 29, len: 1
subject id: 30, len: 1
subject id: 31, len: 1
subject id: 32, len: 1
subject id: 33, len: 1
subject id: 34, len: 1
subject id: 35, len: 1
subject id: 36, len: 1
subject id: 37, len: 1
subject id: 38, len: 1
subject id: 39, len: 1
subject id: 40, len: 1
subject id: 41, len: 1
subject id: 42, len: 1
sub

In [66]:
# combine all client labelled data into one
combined_labelled_data = []
combined_labelled_labels = []
for i in range(num_clients):
    for data, labels in labelled_data[i]:
        combined_labelled_data.append(data)
        combined_labelled_labels.append(labels)
combined_labelled_data = torch.cat(combined_labelled_data, dim=0)
combined_labelled_labels = torch.cat(combined_labelled_labels, dim=0)
# create dataset and dataloader
combined_labelled_dataset = torch.utils.data.TensorDataset(combined_labelled_data, combined_labelled_labels)
combined_labelled_dataloader = torch.utils.data.DataLoader(combined_labelled_dataset, batch_size=batch_size, shuffle=True)

print(f"combined labelled: {len(combined_labelled_dataloader)}")

combined labelled: 255


### Test Data

In [67]:
test_data = []

for i in range(num_clients):
    test = load_data_client(id= i, batch_size=batch_size, type='test')
    print(f"subject id: {i}, len: {len(test)}")
    test_data.append(test)

subject id: 0, len: 14
subject id: 1, len: 14
subject id: 2, len: 15
subject id: 3, len: 13
subject id: 4, len: 16
subject id: 5, len: 16
subject id: 6, len: 4
subject id: 7, len: 4
subject id: 8, len: 3
subject id: 9, len: 14
subject id: 10, len: 11
subject id: 11, len: 15
subject id: 12, len: 13
subject id: 13, len: 16
subject id: 14, len: 14
subject id: 15, len: 12
subject id: 16, len: 16
subject id: 17, len: 14
subject id: 18, len: 14
subject id: 19, len: 14
subject id: 20, len: 14
subject id: 21, len: 14
subject id: 22, len: 12
subject id: 23, len: 14
subject id: 24, len: 2
subject id: 25, len: 2
subject id: 26, len: 2
subject id: 27, len: 2
subject id: 28, len: 2
subject id: 29, len: 2
subject id: 30, len: 2
subject id: 31, len: 2
subject id: 32, len: 2
subject id: 33, len: 2
subject id: 34, len: 2
subject id: 35, len: 2
subject id: 36, len: 2
subject id: 37, len: 2
subject id: 38, len: 2
subject id: 39, len: 2
subject id: 40, len: 2
subject id: 41, len: 2
subject id: 42, len: 2


In [68]:
# load cluster member
cluster_member = np.load(f'Refused_FL/Model_Global_CHAR/2024-02-12_22-06-59/cluster_member.npy')

cluster_member

array([0, 0, 0, 1, 2, 1, 0, 3, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 3, 0, 3, 0, 2, 1, 1, 2, 2, 0, 0, 3, 2, 0, 1, 0, 1, 0, 2, 1,
       3, 3, 0, 0, 3, 2, 2, 0, 0, 2], dtype=int64)

In [69]:
test_loader_clusters = [None for _ in range(n_clusters)]

for c in range(n_clusters):
    combined_test_data = []
    combined_test_labels = []
    for i in range(num_clients):
        if c == cluster_member[i]:
            # Assuming 'test_data[i]' is a DataLoader or similar iterable
            for data, labels in test_data[i]:
                combined_test_data.append(data)
                combined_test_labels.append(labels)
    # Combine the data and labels tensors
    combined_test_data = torch.cat(combined_test_data, dim=0)
    combined_test_labels = torch.cat(combined_test_labels, dim=0)
    # Create dataset and dataloader for the combined data
    combined_test_dataset = torch.utils.data.TensorDataset(combined_test_data, combined_test_labels)
    combined_test_dataloader = torch.utils.data.DataLoader(combined_test_dataset, batch_size=batch_size, shuffle=True)

    # Store the combined dataloader for the current cluster
    test_loader_clusters[c] = combined_test_dataloader

    print(f"Combined test for cluster {c}: {len(combined_test_dataloader)} batches")

Combined test for cluster 0: 172 batches
Combined test for cluster 1: 128 batches
Combined test for cluster 2: 28 batches
Combined test for cluster 3: 14 batches


In [22]:
# # combine all client labelled data into one
# combined_test_data = []
# combined_test_labels = []
# for i in range(num_clients):
#     for data, labels in test_data[i]:
#         combined_test_data.append(data)
#         combined_test_labels.append(labels)
# combined_test_data = torch.cat(combined_test_data, dim=0)
# combined_test_labels = torch.cat(combined_test_labels, dim=0)
# # create dataset and dataloader
# combined_test_dataset = torch.utils.data.TensorDataset(combined_test_data, combined_test_labels)
# combined_test_dataloader = torch.utils.data.DataLoader(combined_test_dataset, batch_size=batch_size, shuffle=True)

# print(f"combined test: {len(combined_test_dataloader)}")

### Class Weight

In [70]:
# Count the frequency of each class
class_counts = torch.zeros(9)  # num_classes should be defined based on your dataset
for _, target in combined_labelled_dataloader:
    class_counts += torch.bincount(target, minlength=9)

# Calculate class weights
class_counts += 1  # Add 1 to each class count to avoid division by zero
c_weight = 1. / class_counts
c_weight = c_weight / c_weight.sum() * num_classes
c_weight = c_weight.to(device)

In [71]:
class_counts

tensor([9.5700e+02, 1.1640e+03, 1.2580e+03, 1.4310e+03, 8.5600e+02, 3.2600e+02,
        1.0000e+00, 8.7100e+02, 1.3050e+03])

In [72]:
c_weight

tensor([4.1402e-03, 3.4039e-03, 3.1496e-03, 2.7688e-03, 4.6287e-03, 1.2154e-02,
        3.9622e+00, 4.5490e-03, 3.0361e-03], device='cuda:0')

## Model

In [99]:
# pretrained_model_path = 'Refused_FL/Model_Global/2024-02-11_22-01-36/global_model_round_199.pth' # model with 200 epochs, 1 local epoch
# model = CNNFeatureExtractor(num_classes=4)

# #load pretrained model
# model.load_state_dict(torch.load(pretrained_model_path))

# # # Freezing layers up to conv3
# # for name, param in model.named_parameters():
# #     if 'conv3' in name:
# #         break
# #     param.requires_grad = False

# # # Unfreeze layers from conv3 onwards
# # unfreeze = False
# # for name, param in model.named_parameters():
# #     if 'conv3' in name:
# #         unfreeze = True
# #     if unfreeze:
# #         param.requires_grad = True

# model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
# model.to(device)

In [137]:
# model = CNNFeatureExtractor(num_classes=num_classes)
# model.to(device)

## Fine-Tuning

In [73]:
# method to test the model and get the accuracy and f1 score
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            y_true.extend(target.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    accuracy = correct / total
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}, F1 Score: {f1}')
    return accuracy, f1

In [74]:
def fine_tune_model(model, train_loader, test_loader, num_epochs=200):
    # Assuming class weights are calculated and provided as `class_weights`
    # class_weights = torch.tensor(c_weight).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)
    
    model.train()
    for epoch in range(num_epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        acc, f1 = test_model(model, test_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {acc}, F1 Score: {f1}')

In [53]:
# fine_tune_model(model.to(device), combined_labelled_dataloader, combined_test_dataloader,num_epochs=100)

In [78]:
for i in range(n_clusters):
    pretrained_model_path = f'Refused_FL/Model_Global_CHAR/2024-02-12_22-06-59/Group_{i}/cluster_model_round_49.pth'
    model = CNNFeatureExtractor(num_classes=num_classes)
    #load pretrained model
    model.load_state_dict(torch.load(pretrained_model_path))

    # Freezing layers up to conv3
    for name, param in model.named_parameters():
        if 'conv3' in name:
            break
        param.requires_grad = False

    # Unfreeze layers from conv3 onwards
    unfreeze = False
    for name, param in model.named_parameters():
        if 'conv3' in name:
            unfreeze = True
        if unfreeze:
            param.requires_grad = True

    model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=9)
    model.to(device)

    print(f"Cluster: {i}")
    print("*" * 50)
    fine_tune_model(model.to(device), train_labeled_loader_clusters[i], test_loader_clusters[i],num_epochs=20)
    print("*" * 50)

Cluster: 0
**************************************************
Accuracy: 0.34356164383561644, F1 Score: 0.2736094718810029
Epoch 1/20, Loss: 1.5322577953338623, Accuracy: 0.34356164383561644, F1 Score: 0.2736094718810029
Accuracy: 0.3795433789954338, F1 Score: 0.3366447229317946
Epoch 2/20, Loss: 1.473305583000183, Accuracy: 0.3795433789954338, F1 Score: 0.3366447229317946
Accuracy: 0.41771689497716896, F1 Score: 0.3682660474660588
Epoch 3/20, Loss: 1.5476433038711548, Accuracy: 0.41771689497716896, F1 Score: 0.3682660474660588
Accuracy: 0.41899543378995435, F1 Score: 0.4087097312904675
Epoch 4/20, Loss: 1.4476304054260254, Accuracy: 0.41899543378995435, F1 Score: 0.4087097312904675
Accuracy: 0.36730593607305934, F1 Score: 0.3502369733707122
Epoch 5/20, Loss: 1.4544806480407715, Accuracy: 0.36730593607305934, F1 Score: 0.3502369733707122
Accuracy: 0.43269406392694065, F1 Score: 0.41273234998497516
Epoch 6/20, Loss: 1.2939770221710205, Accuracy: 0.43269406392694065, F1 Score: 0.412732349

In [141]:
# test_model(model, combined_test_dataloader)

Accuracy: 0.3629350720911011, F1 Score: 0.35672745069023415


(0.3629350720911011, 0.35672745069023415)

In [75]:
clulster_identity = np.load(f'Refused_FL/Model_Global_CHAR/2024-02-12_22-06-59/cluster_member.npy')
cluster_info = [[ idx for idx, g_id in enumerate(clulster_identity) if g_id == i] for i in range(max(clulster_identity)+1)]
for idx, info in enumerate(cluster_info):
    print('Cluster {}: {}'.format(idx, info))

Cluster 0: [0, 1, 2, 6, 8, 12, 14, 16, 17, 18, 20, 22, 23, 25, 27, 33, 34, 37, 39, 41, 46, 47, 51, 52]
Cluster 1: [3, 5, 9, 10, 11, 13, 15, 19, 21, 29, 30, 38, 40, 43]
Cluster 2: [4, 28, 31, 32, 36, 42, 49, 50, 53]
Cluster 3: [7, 24, 26, 35, 44, 45, 48]


## Fine-Tuning per Clusters

In [77]:
train_labeled_loader_clusters = [None for _ in range(n_clusters)]

for c in range(n_clusters):
    combined_label_data = []
    combined_label_labels = []
    for i in range(num_clients):
        if c == cluster_member[i]:
            for data, labels in labelled_data[i]:
                combined_label_data.append(data)
                combined_label_labels.append(labels)
    # Combine the data and labels tensors
    combined_label_data = torch.cat(combined_label_data, dim=0)
    combined_label_labels = torch.cat(combined_label_labels, dim=0)
    # Create dataset and dataloader for the combined data
    combined_label_dataset = torch.utils.data.TensorDataset(combined_label_data, combined_label_labels)
    combined_label_dataloader = torch.utils.data.DataLoader(combined_label_dataset, batch_size=batch_size, shuffle=True)

    # Store the combined dataloader for the current cluster
    train_labeled_loader_clusters[c] = combined_label_dataloader

    print(f"Combined labeled for cluster {c}: {len(combined_label_dataloader)} batches")

Combined labeled for cluster 0: 128 batches
Combined labeled for cluster 1: 98 batches
Combined labeled for cluster 2: 20 batches
Combined labeled for cluster 3: 9 batches
